# OpenCV示例：综合项目示例


In [ ]:
import cv2import numpy as npprint("综合项目示例")


OpenCV示例：综合项目示例学习目标:    1. 文档扫描器实现    2. 颜色识别分类器实现    3. 简单人脸检测实现

In [ ]:
项目1：文档扫描器 ====================print("\n[项目1] 文档扫描器")def document_scanner(img_path=None):

In [ ]:
项目2：颜色识别分类器 ====================print("\n[项目2] 颜色识别分类器")def color_classifier(img_path=None):

In [ ]:
项目3：简单人脸检测 ====================print("\n[项目3] 简单人脸检测")def face_detector(img_path=None):